### Phi-2 (FP16)
#### NOTE: YOU WILL NEED LLAMA-CPP-PYTHON VERSION v0.2.24 OR HIGHER

This notebook demonstrates the use of LlamaIndex for Retrieval Augmented Generation in Linux and with Nvidia's CUDA.

See the [README.md](README.md) file for help on how to run this.

#### 1. Prepare Llama Index for use

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex
from llama_index import SimpleDirectoryReader
from llama_index import ServiceContext

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


#### 2. Load the Word document(s)

Note: A fictitious story about Thundertooth a dinosaur who has travelled to the future. Thanks ChatGPT!

In [2]:
documents = SimpleDirectoryReader("./Data/").load_data()

#### 3. Instantiate the model

In [3]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=None, # We'll load locally.
    model_path='./Models/phi-2-ggml-model-f16.gguf', # FP16 version of Phi-2
    temperature=0.1,
    max_new_tokens=1024,
    context_window=2048, # Phi-2 2K context window - this could be a limitation for RAG as it has to put the content into this context window
    generate_kwargs={},
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 33}, # 33 layers were all that were required and this fit within the RTX 3090's 24GB
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6
llama_model_loader: loaded meta data with 18 key-value pairs and 325 tensors from ./Models/phi-2-ggml-model-f16.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight f16      [  2560, 51200,     1,     1 ]
llama_model_loader: - tensor    1:             blk.0.attn_norm.bias f32      [  2560,     1,     1,     1 ]
llama_model_loader: - tensor    2:           blk.0.attn_norm.weight f32      [  2560,     1,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_qkv.bias f32      [  7680,     1,     1,     1 ]
llama_model_loader: - tensor    4:            blk.0.attn_qkv.weight f16      [  2560,  7680,     1,     1 ]
llama_model_loader: - tensor    5:           blk.0.attn_output.bias f32      [  2560,     1,     1,     1 ]
llama_model_loader:

#### 4. Checkpoint

Are you running on GPU? The above output should include near the top something like:
> ggml_init_cublas: found 1 CUDA devices:

And in the full text near the bottom should be:
> llm_load_tensors: using CUDA for GPU acceleration

#### 5. Embeddings

Convert your source document text into embeddings.

The embedding model is from huggingface, this one performs well.

> https://huggingface.co/thenlper/gte-large


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="thenlper/gte-large", cache_folder=None)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-large
Load pretrained SentenceTransformer: thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Did not find folder thenlper/gte-large
Did not find folder thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Search model on server: http://sbert.net/models/thenlper/gte-large.zip
Search model on server: http://sbert.net/models/thenlper/gte-large.zip
INFO:sentence_transformers.SentenceTransformer:Load SentenceTransformer from folder: /home/mark/.cache/torch/sentence_transformers/sbert.net_models_thenlper_gte-large
Load SentenceTransformer from folder: /home/mark/.cache/torch/sentence_transformers/sbert.net_models_thenlper_gte-large
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda


#### 6. Prompt Template

Prompt template for Phi-2 is below. As there's only a prompt we will combine the system message and prompt into the prompt.

Instruct: {prompt}<br>
Output:

In [5]:
# Produces a prompt specific to the model
def modelspecific_prompt(promptmessage):
    # As per https://huggingface.co/TheBloke/phi-2-GGUF
    return f"Instruct: {promptmessage}\nOutput:"

#### 7. Service Context

For chunking the document into tokens using the embedding model and our LLM

In [6]:
service_context = ServiceContext.from_defaults(
    chunk_size=256, # Number of tokens in each chunk
    # chunk_overlap=1,    # Doesn't allow zero, set to lowest value
    # context_window=2048, # This should be automatically set with the model metadata but we'll force it to ensure wit is
    # num_output=512, # Maximum output from the LLM, let's put this at 512 to ensure LlamaIndex saves that "space" for the output
    llm=llm,
    embed_model=embed_model
)

#### 8. Index documents

In [7]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

#### 9. Query Engine

Create a query engine, specifying how many citations we want to get back from the searched text (in this case 3).

The DB_DOC_ID_KEY is used to get back the filename of the original document

In [8]:
from llama_index.query_engine import CitationQueryEngine
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    # citation_chunk_size=256 #256,
)

# For citations we get the document info
DB_DOC_ID_KEY = "db_document_id"

In [9]:
# Let's look at the chunking and prompting parameters from our service_context

print("service_context.node_parser:")
print("Chunk Overlap:",service_context.node_parser.chunk_overlap)
print("Chunk Size:",service_context.node_parser.chunk_size)
print("Paragraph Separator:",service_context.node_parser.paragraph_separator.replace("\n","\\n"))
print("Secondary Chunking RegEx:",service_context.node_parser.secondary_chunking_regex)
print("Include Metadata:",service_context.node_parser.include_metadata)

print("\nservice_context.prompt_helper:")
print("Context Window:", service_context.prompt_helper.context_window) # The maximum context size that will get sent to the LLM.
print("Chunk Overlap Ratio:", service_context.prompt_helper.chunk_overlap_ratio) # The percentage token amount that each chunk should overlap.
print("Chunk Size Limit:", service_context.prompt_helper.chunk_size_limit) # The maximum size of a chunk.
print("Chunk Separator: '", service_context.prompt_helper.separator, "'") # The separator when chunking tokens.
print("Num Output: '", service_context.prompt_helper.num_output, "'") # The number of tokens that should be left to generate in the prompt (default is 256)

service_context.node_parser:
Chunk Overlap: 200
Chunk Size: 256
Paragraph Separator: \n\n\n
Secondary Chunking RegEx: [^,.;。？！]+[,.;。？！]?
Include Metadata: True

service_context.prompt_helper:
Context Window: 2048
Chunk Overlap Ratio: 0.1
Chunk Size Limit: None
Chunk Separator: '   '
Num Output: ' 1024 '


#### 10. Prompt and Response function

Pass in a question, get a response back.

IMPORTANT: The prompt is set here, adjust it to match what you want the LLM to act like and do.

In [10]:
def RunQuestion(questionText):
    # Excluding the system prompt as the model is including it (even a short version of it) is causing lack of responses in some cases and it is not consistently answering.
    prompt = "" # "You are a story teller who likes to elaborate and answers questions in a positive, helpful and interesting way, so please answer the following question - "
    
    prompt = prompt + questionText

    queryQuestion = modelspecific_prompt(prompt)

    response = query_engine.query(queryQuestion)

    return response

#### 11. Questions to test with

In [11]:
TestQuestions = [
    "Summarise this story for me",
    "Who was the main protagonist?",
    "Did they have any children? If so, what were their names?",
    "Did anything eventful happen?",
]

#### 12. Run Questions through model (this can take a while) and see citations

Runs each test question, saves it to a dictionary for output in the last step.

Note: Citations are the source documents used and the text the response is based on. This is important for RAG so you can reference these documents for the user, and to ensure it's utilising the right documents.

In [12]:
qa_pairs = []

for indexNum, question in enumerate(TestQuestions, start=1):
    question = question.strip() # Clean up

    print(f"\n{indexNum}/{len(TestQuestions)}: {question}")

    response = RunQuestion(question) # Query and get  response

    qa_pairs.append((question.strip(), str(response).strip())) # Add to our output array

    # Displays the citations
    for indexNumCit, node in enumerate(response.source_nodes, start=1):
        print(f"Source {indexNumCit} of {len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")

    # Uncomment the following line if you want to test just the first question
    # break 


1/4: Summarise this story for me


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Source 1 of 3: |Thundertooth Part 3.docx| Source 1:
Thundertooth, ever the protector of his newfound home, wasted no time. With a determined gleam in his eyes, he gathered his family and hurried to the city's command center, where Mayor Grace and the leading scientists were coordinating the evacuation efforts.



The mayor, recognizing Thundertooth's intelligence and resourcefulness, approached him. "Thundertooth, we need a plan to divert or neutralize the meteor. Our technology can only do so much, but with your unique abilities, perhaps we can find a solution."



Thundertooth nodded, understanding the gravity of the situation. He gathered Lumina, Echo, Sapphire, and Ignis, explaining the urgency and the role each of them would play in the impending crisis.



1. **Lumina**: Utilizing her deep understanding of technology, Lumina would enhance the city's energy systems to generate a powerful force field, providing a protective barrier against the meteor's impact.



2. **Echo**: With 


llama_print_timings:        load time =    1340.87 ms
llama_print_timings:      sample time =       0.85 ms /     2 runs   (    0.43 ms per token,  2350.18 tokens per second)
llama_print_timings: prompt eval time =    3941.40 ms /  1150 tokens (    3.43 ms per token,   291.77 tokens per second)
llama_print_timings:        eval time =      81.40 ms /     1 runs   (   81.40 ms per token,    12.28 tokens per second)
llama_print_timings:       total time =    4036.72 ms


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...


llama_print_timings:        load time =    1340.87 ms
llama_print_timings:      sample time =     305.11 ms /  1009 runs   (    0.30 ms per token,  3307.06 tokens per second)
llama_print_timings: prompt eval time =    2701.05 ms /   761 tokens (    3.55 ms per token,   281.74 tokens per second)
llama_print_timings:        eval time =   94349.63 ms /  1008 runs   (   93.60 ms per token,    10.68 tokens per second)
llama_print_timings:       total time =  100767.20 ms


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit


Source 1 of 3: |Thundertooth Part 3.docx| Source 1:
Thundertooth stood at the forefront, using his mighty roar to coordinate and inspire the efforts of the city's inhabitants. The ground trembled as the meteor drew closer, but the Thundertooth family's coordinated efforts began to take effect. Lumina's force field shimmered to life, deflecting the meteor's deadly path. Echo's amplified warnings reached every corner of the city, ensuring that no one was left behind.



As Ignis's controlled bursts of flames interacted with the meteor, it began to change course. The combined efforts of the Thundertooth family, guided by their unique talents, diverted the catastrophic collision. The meteor, once destined for destruction, now harmlessly sailed past the Earth, leaving the city and its inhabitants unscathed.



The citizens, emerging from their shelters, erupted into cheers of gratitude. Mayor Grace approached Thundertooth, expressing her heartfelt thanks for the family's heroic efforts. The


llama_print_timings:        load time =    1340.87 ms
llama_print_timings:      sample time =     294.68 ms /   968 runs   (    0.30 ms per token,  3284.97 tokens per second)
llama_print_timings: prompt eval time =    2850.26 ms /   802 tokens (    3.55 ms per token,   281.38 tokens per second)
llama_print_timings:        eval time =   91916.21 ms /   967 runs   (   95.05 ms per token,    10.52 tokens per second)
llama_print_timings:       total time =   98386.46 ms


#### 13. Output responses

In [13]:
for indexQA, (question, answer) in enumerate(qa_pairs, start=1):
    print(f"{indexQA}/{len(qa_pairs)} {question}\n\n{answer}\n\n--------\n")

1/4 Summarise this story for me



--------

2/4 Who was the main protagonist?

The main protagonist is Thundertooth.

--------

3/4 Did they have any children? If so, what were their names?

[/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] 